In [49]:
staff = {"design": 20,
"uiux": 5,
"frontend": 10,
"data": 15,
"backend": 5,
"account": 20,
"maint": 1,
"exec": 10,
"ceo": 1,
"dog": 1
}

In [6]:
import urllib.parse
url = "https://api.foursquare.com/v3/places/search?"
parameters = {
    "ll": ",".join(map(str,[41.415730,2.1643247])),
    "radius" : 30000,
    "sort": "DISTANCE"
}

parameters['categories'] =  ",".join(map(str,[19031,19040]))

url = url + urllib.parse.urlencode(parameters)
url

'https://api.foursquare.com/v3/places/search?ll=41.41573%2C2.1643247&radius=30000&sort=DISTANCE&categories=19031%2C19040'

In [9]:
from src.criteria import conditions
conditions

{1: {'max_num': 10,
  'max_dist': 5,
  'people': 20,
  'description': 'Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.',
  'source': 'Crunchbase',
  'categories': ['tags_list', 'description', 'name'],
  'query': 'design'},
 2: {'max_num': 5,
  'max_dist': 3,
  'people': 26,
  'description': '30% of the company staff have at least 1 child.',
  'source': 'Foursquare',
  'categories': [12055, 12056, 12057, 12058, 12059, 12060, 12061, 12062],
  'query': None},
 3: {'max_num': 20,
  'max_dist': 20,
  'people': 15,
  'description': 'Developers like to be near successful tech startups that have raised at least 1 Million dollars.',
  'source': 'Crunchbase',
  'categories': 'total_money_raised',
  'query': 'More than 1,000,000'},
 4: {'max_num': 1,
  'max_dist': 2,
  'people': 10,
  'description': "Executives like Starbucks A LOT. Ensure there's a starbucks not too far.",
  'source': 'Foursquare',
  'categories': [13032],
  'que

In [130]:
import src.criteria as cr

In [10]:
load_dotenv()

NameError: name 'load_dotenv' is not defined

In [131]:
cr.staff

{'design': 20,
 'uiux': 5,
 'frontend': 10,
 'data': 15,
 'backend': 5,
 'account': 20,
 'maint': 1,
 'exec': 10,
 'ceo': 1,
 'dog': 1}

In [50]:
conditions = { 
            0: {
                "max_num": 10,
                "max_dist": 5,
                "people": staff["design"]
                "description": "Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design."
                "source": "Crunchbase",
                "categories": ["tags_list","description","name"]
                "query": "design"
            }
    
            1: {
                "max_num": 5,
                "max_dist": 3,
                "people": round(sum(staff.values())*0.3),
                "description": "30% of the company staff have at least 1 child.",
                "source": "Foursquare",
                "categories": [12055, 12056, 12057, 12058, 12059, 12060, 12061, 12062],
                "query": None
             },

              2: {
                  "max_num": 20,
                  "max_dist": 20,
                  "people": staff["frontend"] + staff["backend"],
                  "description": "Developers like to be near successful tech startups that have raised at least 1 Million dollars.",
                  "source": "Crunchbase",
                  "categories": "total_money_raised"
                  "query": "More than 1,000,000"
              },
              3: {
                  "max_num": 1,
                  "max_dist": 2,
                  "people": staff["exec"],
                  "description": "Executives like Starbucks A LOT. Ensure there's a starbucks not too far."
                  "source": "Foursquare",
                  "category":[13032]
                  "query": "Starbucks"
                  
              },
              4: {
                  "max_num": 1,
                  "max_dist": 50,
                  "people": staff["account"],
                  "description": "Account managers need to travel a lot."
                  "source": "Foursquare",
                  "category":[19031, 19040]
                  "query": None
              },
              5: {"max_num": 30,
                  "max_dist": 3,
                  "people": sum(staff.values()),
                  "description": "Everyone in the company is between 25 and 40, give them some place to go party."
                  "source": "Foursquare",
                  "category":[13003, 10032, ]
                  "query": None
              },
              6: {
                  "max_num": 2,
                  "max_dist": 2,
                  "people": staff["ceo"],
                  "description":"The CEO is vegan.",
                  "source": "Foursquare",
                  "category":[13377]
                  "query": None
              },
              7: {
                  "max_num": 1,
                  "max_dist": 10,
                  "people": staff["maint"],
                  "description": "If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km."
                  "source": "Foursquare",
                  "category":[18008]
                  "query": None
              },
              8: {
                  "max_num": 1,
                  "max_dist": 4,
                  "people": staff["dog"],
                  "description": "The office dog Dobby needs a hairdresser every month. Ensure there's one not too far away."
                  "source": "Foursquare",
                  "category":[11134]
                  "query": None
              },
}

In [51]:
def score_calculator(condition, num, min_dist):
    if num > 0:
        distance = 1 / min_dist
        if num > conditions[condition]['max_num']:
            number = 1
        else:
            number = num / conditions[condition]['max_num']
        score = (number + distance)*conditions[condition]["people"]
    else:
        score = 0
    return score

In [132]:
import pandas as pd
pd.DataFrame(cr.conditions)

,1,2,3,4,5,6,7,8,9
max_num,10,5,20,1,1,30,2,1,1
max_dist,5,3,20,2,50,3,2,10,4
people,20,26,15,10,20,87,1,1,1
description,Designers like to go to design talks and share...,30% of the company staff have at least 1 child.,Developers like to be near successful tech sta...,Executives like Starbucks A LOT. Ensure there'...,Account managers need to travel a lot.,"Everyone in the company is between 25 and 40, ...",The CEO is vegan.,"If you want to make the maintenance guy happy,...",The office dog Dobby needs a hairdresser every...
source,Crunchbase,Foursquare,Crunchbase,Foursquare,Foursquare,Foursquare,Foursquare,Foursquare,Foursquare
categories,"[tags_list, description, name]","[12055, 12056, 12057, 12058, 12059, 12060, 120...",total_money_raised,[13032],"[19037, 19040]",NaN,NaN,NaN,NaN
query,design,None,"More than 1,000,000",Starbucks,None,None,None,None,None
category,NaN,NaN,NaN,NaN,NaN,"[13003, 10032]",[13377],[18008],[11134]


## cond 2

In [108]:
from pymongo import MongoClient
import pandas as pd
import time
client = MongoClient("localhost:27017")
db=client['Ironhack']
c =db.get_collection('lab')


In [112]:
query = {"total_money_raised":{"$exists":"true"}}
projection = {"name":1, "total_money_raised":1, "offices":1, "_id":0}
companies = list(c.find(query, projection))

In [113]:
def clean_money (string):
    '''
    Function to convert to an aomount in USD the string located in the field total_money_raised of the Crunchbase database.
    If can't be converted prints the string
    '''
    money = ""
     # Currency convertion value 29/10/2022
    if "C$" in string:
        multi = 0.73
        string = string.replace("C$", "")
    elif "$" in string:
        multi = 1
        string = string.replace("$", "")
    elif "€" in string:
        multi = 1
        string = string.replace("€", "")
    elif "£" in string:
        multi = 1.16
        string = string.replace("£", "")
    elif "¥" in string:    
        multi = 0.0068 #All companies that contain ¥ are located in Japan
        string = string.replace("¥", "")
    elif "kr" in string:
        multi = 0.091
        string = string.replace("kr", "")
    # Transforming string to a number
    for i in string:
            if i.isdigit() or i== ".":
                money = money + i
            elif i == 'K' or i == 'k':
                multi = 1000
            elif i == 'M' or i == 'm':
                multi = 1000000
            elif i == 'B' or i == 'b':
                multi = 1000000000
            else:
                print(string)
                return (string)
    return float(money)*multi
        

In [114]:
companies_df = pd.DataFrame(companies)
companies_df.dropna(inplace=True)

In [115]:
companies_df["total_money_raised"] = companies_df["total_money_raised"].apply(clean_money)
companies_df = companies_df[companies_df["total_money_raised"] > 1000000]

In [116]:
companies_df.shape

(3922, 3)

In [118]:
offices_df = []
for i, v in companies_df.iterrows():
    for e in v["offices"]:
        office = {}
        office['name'] = v['name']
        office['country_code'] = e['country_code']
        office['city'] = e['city']
        office['latitude'] = e['latitude']
        office['longitude'] = e['longitude']
        offices_df.append(office)
    

In [119]:
offices_df = pd.DataFrame(offices_df)

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494
...,...,...,...,...,...
5071,Celestial Semiconductor,USA,San Jose,37.386417,-121.929170
5072,Purfresh,USA,Menlo Park,37.477008,-122.151599
5073,InSound Medical,USA,Newark,37.512038,-121.998513
5074,Getyoo,BEL,Brussels,50.839970,4.346472


In [127]:
offices_df=offices_df[offices_df['latitude'].notna()]

In [133]:
offices_df

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494
...,...,...,...,...,...
5069,Cantimer,USA,Menlo Park,37.484616,-122.181753
5071,Celestial Semiconductor,USA,San Jose,37.386417,-121.929170
5072,Purfresh,USA,Menlo Park,37.477008,-122.151599
5073,InSound Medical,USA,Newark,37.512038,-121.998513


In [140]:
import requests
url = "https://api.foursquare.com/v3/places/search?ll=41.415730%2C2.1643247&radius=30000&categories=19031%2C%2019040&sort=RELEVANCE"

headers = {
        "accept": "application/json",
        "Authorization": "fsq3leL0i1Fmvl8L65VPj39HoEfQ77Ypc1/YJefpOvGqM8c="
    }
response = requests.get(url, headers=headers)

In [141]:
response

<Response [200]>

In [142]:
response.json()

{'results': [{'fsq_id': '4b7980a2f964a5202bfd2ee3',
   'categories': [{'id': 19031,
     'name': 'Airport',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 15237,
   'geocodes': {'main': {'latitude': 41.289084, 'longitude': 2.073963},
    'roof': {'latitude': 41.289084, 'longitude': 2.073963}},
   'link': '/v3/places/4b7980a2f964a5202bfd2ee3',
   'location': {'admin_region': 'Cataluña',
    'country': 'ES',
    'formatted_address': '08820 El Prat de Llobregat Catalonia',
    'locality': 'El Prat de Llobregat',
    'postcode': '08820',
    'region': 'Catalonia'},
   'name': 'Barcelona–El Prat Josep Tarradellas Airport (BCN) (Aeropuerto Josep Tarradellas Barcelona-El Prat)',
   'related_places': {'children': [{'fsq_id': '4b9a0afaf964a5204d9c35e3',
      'name': 'RENFE Aeroport'},
     {'fsq_id': '4cea74b8e888f04d743b4d6b', 'name': 'Terminal 2B'},
     {'fsq_id': '4b80be31f964a5209a8930e3', 'name'

In [1]:
import src.queries as qu
location = (41.289084, 2.073963)


In [2]:
import time

In [3]:
KP

NameError: name 'KP' is not defined

In [4]:
FP

NameError: name 'FP' is not defined

In [2]:
print(qu.FP)

fsq3leL0i1Fmvl8L65VPj39HoEfQ77Ypc1/YJefpOvGqM8c=


In [10]:
import os
from dotenv import load_dotenv
env_path = (os.path.join("", ".env"))
load_dotenv(env_path)

True

In [3]:
qu.foursquare(location,2)

<Response [200]>

In [4]:
locations = qu.foursquare(location,4)

In [5]:
locations

<Response [200]>

In [8]:
locations.json()

{'results': [{'fsq_id': '5bb76b2c3731ee002c967160',
   'categories': [{'id': 13035,
     'name': 'Coffee Shop',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
      'suffix': '.png'}}],
   'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
     'name': 'Starbucks'}],
   'distance': 152,
   'geocodes': {'main': {'latitude': 41.290455, 'longitude': 2.073936}},
   'link': '/v3/places/5bb76b2c3731ee002c967160',
   'location': {'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': '08820 El Prat de Llobregat Catalonia',
    'locality': 'El Prat de Llobregat',
    'postcode': '08820',
    'region': 'Catalonia'},
   'name': 'Starbucks',
   'related_places': {'parent': {'fsq_id': '4b80be31f964a5209a8930e3',
     'name': 'Terminal 1'}},
   'timezone': 'Europe/Madrid'}],
 'context': {'geo_bounds': {'circle': {'center': {'latitude': 41.289084,
     'longitude': 2.073963},
    'radius': 2000}}}}

In [4]:
from src.database import companies_1m

In [5]:
offices_df = companies_1m(save=True)

In [6]:
offices_df

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494
...,...,...,...,...,...
3248,Cantimer,USA,Menlo Park,37.484616,-122.181753
3249,Celestial Semiconductor,USA,San Jose,37.386417,-121.929170
3250,Purfresh,USA,Menlo Park,37.477008,-122.151599
3251,InSound Medical,USA,Newark,37.512038,-121.998513


In [3]:
offices_df.dropna(inplace=True)

NameError: name 'offices_df' is not defined

In [5]:
offices_df

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494
...,...,...,...,...,...
5069,Cantimer,USA,Menlo Park,37.484616,-122.181753
5071,Celestial Semiconductor,USA,San Jose,37.386417,-121.929170
5072,Purfresh,USA,Menlo Park,37.477008,-122.151599
5073,InSound Medical,USA,Newark,37.512038,-121.998513


In [1]:
from src.database import companies_design

In [2]:
companies_design(save=True)

,name,country_code,city,latitude,longitude
0,Digg,USA,San Francisco,37.764726,-122.394523
1,Adobe Systems,USA,San Jose,37.320052,-121.877636
2,Negonation,ESP,,40.463667,-3.749220
3,GENWI,USA,Los Altos,33.817100,-111.903500
4,BrightQube,USA,Carlsbad,33.095680,-117.244034
...,...,...,...,...,...
493,Top Rated Website Designers,USA,San Diego,33.141212,-117.320512
494,United On-Line,GRC,"Galatsi, Attiki",38.008825,23.759331
495,ITERNOVA,ESP,Teruel,40.340133,-1.105961
496,ITERNOVA,ESP,Zaragoza,41.669878,-0.894487


In [35]:
import pandas as pd
airports = pd.read_csv("./data/airports_raw.csv")

FileNotFoundError: [Errno 2] No such file or directory: './data/irports.csv'

In [29]:
airports = airports[(~airports.iata_code.isna())&(airports.type != "closed")&(airports.type != "heliport")&(airports.type != "seaplane_base")]

In [31]:
airports.columns

Index(['id', 'ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'elevation_ft', 'continent', 'iso_country', 'iso_region',
       'municipality', 'scheduled_service', 'gps_code', 'iata_code',
       'local_code', 'home_link', 'wikipedia_link', 'keywords'],
      dtype='object')

In [36]:
airports = airports[["name", "iata_code", "iso_country","latitude_deg", "longitude_deg"]]

In [37]:
airports.reset_index(drop=True, inplace=True)
airports.rename(columns={'latitude_deg': 'latitude',
                   'longitude_deg': 'longitude'},
          inplace=True, errors='raise')

In [39]:
airports

,name,iata_code,iso_country,latitude,longitude
0,Utirik Airport,UTK,MH,11.222000,169.852005
1,Ocean Reef Club Airport,OCA,US,25.325399,-80.274803
2,Pilot Station Airport,PQS,US,61.934601,-162.899994
3,Crested Butte Airpark,CSE,US,38.851918,-106.928341
4,LBJ Ranch Airport,JCY,US,30.251801,-98.622498
...,...,...,...,...,...
8631,Dalian Zhoushuizi International Airport,DLC,CN,38.965698,121.539001
8632,Tonghua Sanyuanpu Airport,TNH,CN,42.253889,125.703333
8633,Shenyang Taoxian International Airport,SHE,CN,41.639801,123.483002
8634,Yanji Chaoyangchuan Airport,YNJ,CN,42.882801,129.451004


In [1]:
from src.database import airports

In [2]:
airports(save=True)

,name,iata_code,iso_country,latitude,longitude
0,Utirik Airport,UTK,MH,11.222000,169.852005
1,Ocean Reef Club Airport,OCA,US,25.325399,-80.274803
2,Pilot Station Airport,PQS,US,61.934601,-162.899994
3,Crested Butte Airpark,CSE,US,38.851918,-106.928341
4,LBJ Ranch Airport,JCY,US,30.251801,-98.622498
...,...,...,...,...,...
8631,Dalian Zhoushuizi International Airport,DLC,CN,38.965698,121.539001
8632,Tonghua Sanyuanpu Airport,TNH,CN,42.253889,125.703333
8633,Shenyang Taoxian International Airport,SHE,CN,41.639801,123.483002
8634,Yanji Chaoyangchuan Airport,YNJ,CN,42.882801,129.451004


In [2]:
from src.database import offices_list

In [3]:
offices = offices_list(save=True)

In [4]:
offices

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,AdventNet,USA,Pleasanton,37.692934,-121.904945
3,Zoho,USA,Pleasanton,37.692934,-121.904945
4,Digg,USA,San Francisco,37.764726,-122.394523
...,...,...,...,...,...
10738,QSGI,USA,Palm Beach,26.705331,-80.041395
10739,Oriact,CHE,KÃ¼ssnacht,47.088219,8.437163
10740,AfterLogic,USA,Livingston,40.793024,-74.323554
10741,EnteGreat Solutions,USA,Birmingham,33.518885,-86.816068
